# **Goal** :
This notebook aims at producing TOD in a very simplified manner : project the QUBIC beam on the sky, take the value of the pixel, and add it to the TOD. 
This simplified TOD production will be used to test the convergence issue of the TOD with Nsub, and in particular to test our hypothesis about the pixellisation effect.

# Importation

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import healpy as hp

from tqdm import tqdm

from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Instrument.Qacquisition import QubicMultiAcquisitions, compute_freq
from qubic.lib.Qsamplings import get_pointing, equ2gal
from qubic.lib.Qscene import QubicScene
from qubic.lib.MapMaking.FrequencyMapMaking.Qspectra_component import CMBModel

from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator

%matplotlib inline

In [ ]:
nside = 128

nreal = 20

# Initialize QUBIC instance

Build the QUBIC dictionary, mandatory to use the QUBIC functions

In [ ]:
# Build MPI communicator and qubic dict
dictfilename = 'qubic/qubic/dicts/pipeline_demo.dict'
d = qubicDict()
d.read_from_file(dictfilename)

center = equ2gal(d['RA_center'], d['DEC_center'])

d['nf_recon'] = 2
d['MultiBand'] = True
d['nside'] = nside

d['npointings'] = 20
d['synthbeam_kmax'] = 1
d['synthbeam_fraction'] = 1

d['noiseless'] = True
d['photon_noise'] = False
d['use_synthbeam_fits_file'] = False

In [ ]:
# Build scanning strategy
sampling = get_pointing(d)

# Build scene
scene = QubicScene(d)

In [ ]:
# Number of sub-acquisitions
nsub_max = 84
nsub_list = np.arange(4, nsub_max + 1, 8)

In [ ]:
# Build acquisition operator
Qacq = QubicMultiAcquisitions(d, 4, 2, sampling=sampling)

# Use Qacq to save the userful parameters
horn_spacing = Qacq.multiinstrument[0].horn.spacing
horn_angle = Qacq.multiinstrument[0].horn.angle
detectors_position =  Qacq.multiinstrument[0].detector.center

# Build the synthesised beam resolution
fwhm_synthbeam150 = Qacq.multiinstrument[0].synthbeam.peak150.fwhm

# Compute peaks position on the sky & Build TOD from them

In [ ]:
nreal_tod_w_interp = []
nreal_tod_wo_interp = []

for ireal in range(nreal):
    peaks_index_list = []
    tod_list_wo_interp = []
    tod_list_w_interp = []
    freq_list = []
    fwhm_list = []
    bandwidth_list = []

    # Build CMB map
    cl_cmb = CMBModel(None).give_cl_cmb(r=0, Alens=1)
    cmb_map = hp.synfast(cl_cmb, nside, new=True, verbose=False).T

    for nsub in tqdm(nsub_list):
        peaks_index = []
        tod_w, tod_wo = [], []
        
        _, _, filter_nus150, deltas150, _, _ = compute_freq(
                150, int(nsub / 2), relative_bandwidth=d["filter_relative_bandwidth"], frequency_spacing="log")
        _, _, filter_nus220, deltas220, _, _ = compute_freq(
                220, int(nsub / 2), relative_bandwidth=d["filter_relative_bandwidth"], frequency_spacing="log")
        
        nus = np.concatenate((filter_nus150, filter_nus220)) * 1e9
        bandwidth = np.concatenate((deltas150, deltas220))
        fwhm = fwhm_synthbeam150 * 150e9 / nus
        for isub in range(nsub):
            theta, phi = Qacq.multiinstrument[0]._peak_angles_kmax(
                d['synthbeam_kmax'],
                horn_spacing,
                horn_angle,
                nus[isub],
                detectors_position,
            )
            # print('Frequency: ', nus[isub], 'Theta: ', theta, 'Phi: ', phi)

            peaks_index.append(hp.ang2pix(nside, theta, phi))
            ConvOp = HealpixConvolutionGaussianOperator(fwhm[isub])
            
            # I only take the intensity and then, I sum over the peaks values
            value_interp = hp.get_interp_val((cmb_map[..., 0]), theta, phi)#, nest=True)
            value = cmb_map[hp.ang2pix(nside, theta, phi)][..., 0]
            
            tod_w.append(np.sum(value_interp, axis = -1))
            tod_wo.append(np.sum(value, axis = -1))
            
        peaks_index_list.append(np.array(peaks_index))
        tod_list_w_interp.append(np.array(tod_w))
        tod_list_wo_interp.append(np.array(tod_wo))
        freq_list.append(nus/1e9)
        fwhm_list.append(fwhm)
        bandwidth_list.append(bandwidth)
        
    nreal_tod_w_interp.append(tod_list_w_interp)
    nreal_tod_wo_interp.append(tod_list_wo_interp)
    
print("Shape nreal tod list : (Nreal), (len(nsub_list)), (nsub, npix)")

In [ ]:
# Plot the projection of the peaks on the sky
test = np.zeros(hp.nside2npix(nside))

for i in range(np.shape(peaks_index_list[-1])[0]):
    test[peaks_index_list[-1][i, 0]] = i

colors = plt.cm.jet(np.linspace(0, 1, nsub))
hp.mollview(test, title=f"Peaks pointing on the sky - Nside = {nside}", cmap=plt.cm.colors.ListedColormap(colors), unit="Nsub index")
hp.gnomview(test, reso=15, rot=[0, 100], title=f"Peaks pointing on the sky - Nside = {nside}", cmap=plt.cm.colors.ListedColormap(colors), unit="Nsub index")

# Plot TOD

In [ ]:
plt.figure(figsize=(10, 6))

idet = 0
ireal = 0

for isub in range(len(nsub_list)):
    color = plt.cm.jet(isub/len(nsub_list))
    plt.plot(freq_list[isub], nreal_tod_w_interp[ireal][isub][:, idet], 'x', color=color, label=f"Nsub = {nsub_list[isub]}")
    plt.hlines(np.mean(nreal_tod_w_interp[ireal][isub][:, idet]), xmin=131.25, xmax=245, color=color, linestyle='--', linewidth=1)
    
plt.title(f"TOD - Nside = {nside} - Detector index = {idet} - Interpolated Sky = True")
plt.xlabel("Frequency (GHz)")
plt.ylabel("TOD")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlim(131.25, 245)

In [ ]:
plt.figure(figsize=(10, 6))

idet = 0
ireal = 0

for isub in range(len(nsub_list)):
    color = plt.cm.jet(isub/len(nsub_list))
    plt.plot(freq_list[isub], nreal_tod_wo_interp[ireal][isub][:, idet], 'x', color=color, label=f"Nsub = {nsub_list[isub]}")
    plt.hlines(np.mean(nreal_tod_wo_interp[ireal][isub][:, idet]), xmin=131.25, xmax=245, color=color, linestyle='--', linewidth=1)
    
plt.title(f"TOD - Nside = {nside} - Detector index = {idet} - Interpolated Sky = False")
plt.xlabel("Frequency (GHz)")
plt.ylabel("TOD")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlim(131.25, 245)

Be careful, when I sum over the sub-tods, I have to multiply by the bandwidth of the sub-bands

In [ ]:
rel_diff_w_list = []
rel_diff_wo_list = []

for ireal in range(nreal):
    rel_diff_w, rel_diff_wo = [], []
    for insub in range(len(nsub_list)):
        rel_diff_w.append((np.sum(nreal_tod_w_interp[ireal][insub][:, idet]*bandwidth_list[insub], axis=0) - np.sum(nreal_tod_w_interp[ireal][-1][:, idet]*bandwidth_list[-1], axis=0)) / np.sum(nreal_tod_w_interp[ireal][-1][:, idet]*bandwidth_list[-1], axis=0))
        rel_diff_wo.append((np.sum(nreal_tod_wo_interp[ireal][insub][:, idet]*bandwidth_list[insub], axis=0) - np.sum(nreal_tod_wo_interp[ireal][-1][:, idet]*bandwidth_list[-1], axis=0)) / np.sum(nreal_tod_wo_interp[ireal][-1][:, idet]*bandwidth_list[-1], axis=0))        
    rel_diff_w_list.append(rel_diff_w)
    rel_diff_wo_list.append(rel_diff_wo)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

for i in range(len(nsub_list)):    
    ax1.plot(nsub_list[i], np.abs(np.sum(rel_diff_w_list, axis=0)[i]), '.r', label='With interpolation' if i == 0 else '')
    ax2.plot(nsub_list[i], np.abs(np.sum(rel_diff_w_list, axis=0)[i]), '.r', label='With interpolation' if i == 0 else '')
    
    ax1.plot(nsub_list[i], np.abs(np.sum(rel_diff_wo_list, axis=0)[i]), '.b', label='Without interpolation' if i == 0 else '')
    ax2.plot(nsub_list[i], np.abs(np.sum(rel_diff_wo_list, axis=0)[i]), '.b', label='Without interpolation' if i == 0 else '')

for ax in [ax1, ax2]:
    ax.set_xlabel('Nsub')
    ax.set_ylabel(r'|$\frac{H(map)^{max} - H(map)^{i}}{H(map)^{max}}$|')
    ax.legend()

ax2.set_yscale('log')
fig.suptitle(f"TODs' convergence - Average over {nreal} realisations - Nside = {nside}")
plt.tight_layout()
plt.show()